In [ ]:

import os

exe_list = ['wc', 'gvpr']
for sub_directory, directories_list, files_list in os.walk(r'C:\\'):
    for file_name in files_list:
        if any(map(lambda x: file_name == f'{x}.exe', exe_list)):
            file_path = os.path.join(sub_directory, file_name)
            print(f'{file_path}')

C:\\OSGeo4W64\apps\msys\bin\wc.exe
C:\\Program Files\Git\usr\bin\wc.exe
C:\\Program Files (x86)\Gow\bin\wc.exe
C:\\Program Files (x86)\Microsoft Visual Studio\2017\Community\Common7\IDE\CommonExtensions\Microsoft\TeamFoundation\Team Explorer\Git\usr\bin\wc.exe
C:\\Program Files (x86)\Microsoft Visual Studio\2019\Community\Common7\IDE\CommonExtensions\Microsoft\TeamFoundation\Team Explorer\Git\usr\bin\wc.exe
C:\\rtools40\usr\bin\wc.exe
C:\\Users\dev\AppData\Local\GitHubDesktop\app-2.6.1\resources\app\git\usr\bin\wc.exe
C:\\Users\dev\AppData\Local\GitHubDesktop\app-2.6.2\resources\app\git\usr\bin\wc.exe
C:\\Users\dev\AppData\Local\SquirrelTemp\tempb\lib\net45\resources\app\git\usr\bin\wc.exe
C:\\Users\dev\AppData\Local\SquirrelTemp\tempd\lib\net45\resources\app\git\usr\bin\wc.exe


In [ ]:

%run ../load_magic/storage.py
%run ../load_magic/dataframes.py
%run ../load_magic/paths.py
%run ../load_magic/lists.py
%run ../load_magic/environment.py
%pprint
notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])
[fn for fn in dir() if not fn.startswith('_')]